# Data and imports


In [1]:
import polars as pl
import unicodedata
from pathlib import Path
import html
import re
from transformers.pipelines import pipeline
from sentence_transformers import SentenceTransformer
from keyphrase_vectorizers import KeyphraseCountVectorizer, KeyphraseTfidfVectorizer
import nltk
from nltk.corpus import stopwords
import torch, platform
import random
import numpy as np
from keybert import KeyBERT
import pymorphy3
import spacy


# Path
PATH_TO_ITEMS = Path().cwd().parent / "data" / "modified_data" / "items.parquet"
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rustemhutiev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# For MacOs to check if MPS is available

print("macOS:", platform.mac_ver()[0], "  PyTorch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built‑in:", torch.backends.mps.is_built())

macOS: 15.4.1   PyTorch: 2.7.0
MPS available: True
MPS built‑in: True


In [3]:
# Specifing seed value for reproducibility

SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
pl.set_random_seed(SEED)


In [4]:
items_df = pl.read_parquet(PATH_TO_ITEMS)

# Checking if there is any null value in the DataFrame
null_sum = items_df.null_count().sum_horizontal()[0]

assert null_sum == 0, f"There are {null_sum} null values in the DataFrame"


# Clustering


## Keywords problem


- Проверим, насколько наши текущие ключевые слова подходят для кластеризации


In [5]:
# Print samples of dataset's keywords
sample_dataset = (
    items_df.sample(2)
    .select("keywords", "countries", "title", "description")
    .rows(named=True)
)

for item in sample_dataset:
    print(f"Title: {item['title']}")
    print(f"Countries: {item['countries']}")
    print(f"Keywords: {item['keywords'][:150]}...")
    print("-" * 80)

Title: с прицепом
Countries: сша
Keywords: прицепом, 2017, США...
--------------------------------------------------------------------------------
Title: виктория
Countries: великобритания
Keywords: Виктория, 2016, Великобритания, брак, короли, королевы, коррупция, отцы, дети, политика, свадьбы, семейные, проблемы, семья, отношения, отношения, муж...
--------------------------------------------------------------------------------


- Во-первых, идет дубляция страны, что не хорошо. Зачем нам заниматься увеличением токенов, которые у нас повторяются в графе Стран
- Видно, что предложения по типу "борьба за выживание" разделены запятой, а не точкой с запятой

- Необходимо придумать более осмысленные ключевые слова!


### Bert for keywords


In [6]:
# Model

# If mac, i am using Mac for this example
device = "mps" if torch.backends.mps.is_available() else "cpu"

# If you have gpu uncomment the line below
# device = "cuda" if torch.cuda.is_available() else device

model = SentenceTransformer("cointegrated/rubert-tiny2", device=device)

kw_model = KeyBERT(model)  # type: ignore

In [7]:
# stop_words if needed for analysis

nlp = spacy.load("ru_core_news_sm")

stopwords = nlp.Defaults.stop_words
morph = pymorphy3.MorphAnalyzer()

In [16]:
vectorizer = KeyphraseCountVectorizer(
    spacy_pipeline="ru_core_news_sm",
    pos_pattern=r"(<ADJ.*>*<NOUN.*>+|<NOUN.*>+<VERB.*>+|<NOUN.*>+<ADP.*>*<NOUN.*>+)",
    stop_words=stopwords,
    spacy_exclude=["textcat"],
)


def preclean(text: str) -> str:
    text = html.unescape(text)  # &amp; → &
    text = unicodedata.normalize("NFKC", text)  # длинные тире → обычные
    text = re.sub(r"<[^>]+>", " ", text)  # убираем HTML
    text = re.sub(r"\d{4}", " ", text)  # опц.: убираем года
    text = re.sub(r"[^\S\n]+", " ", text)  # множественные пробелы
    return text.strip()


def extract_cleaned_kw(text: str) -> str:
    clean_text = preclean(text)

    extracted_keywords_with_scores = kw_model.extract_keywords(
        clean_text,  # Corrected from 'text' to 'clean_text'
        use_mmr=True,
        vectorizer=vectorizer,
    )

    keyword_phrases = [phrase for phrase, score in extracted_keywords_with_scores]

    return ", ".join(keyword_phrases)

In [18]:
from tqdm.notebook import tqdm

descriptions = items_df["description"].to_list()

keywords = [
    extract_cleaned_kw(desc) for desc in tqdm(descriptions, desc="Extracting keywords")
]

items_df = items_df.with_columns(pl.Series("keywords", keywords))

Extracting keywords:   0%|          | 0/15963 [00:00<?, ?it/s]

KeyboardInterrupt: 